In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import os
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import math
import glob
import random
from PIL import Image
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
IMG_FOLDER = "/content/drive/My Drive/HVD_APP/data/"
#IMG_FOLDER = "/content/drive/My Drive/PD/output3/"
folders_all = glob.glob(IMG_FOLDER + "/*")

print(len(folders_all))

folders_valid = random.sample(folders_all,300)

folders = [folder for folder in folders_all if folder not in folders_valid]
random.shuffle(folders)

def rand_img():
  cc=0
  while cc<len(folders):
    folder = folders[cc]
    y = random.randint(0,1)
    imgs = np.zeros((128*2, 128*2))
    flip = random.random()
    img_front = np.array(Image.open(folder + "/front.tif"))
    img_back =np.array(Image.open(folder + "/back.tif"))
    img_left =np.array(Image.open(folder + "/left.tif"))
    img_right =np.array(Image.open(folder + "/right.tif"))
    if flip> 0.5:
      imgs[0:128, 0*128:(0+1)*128] = np.flip(img_front, axis = 1)
      imgs[0:128, 1*128:(1+1)*128] = np.flip(img_back, axis = 1)
      imgs[128:256, 0*128:(0+1)*128] = np.flip(img_right, axis = 1)
      imgs[128:256, 1*128:(1+1)*128] = np.flip(img_left, axis = 1)
    else:
      imgs[0:128, 0*128:(0+1)*128] = img_front
      imgs[0:128, 1*128:(1+1)*128] = img_back
      imgs[128:256, 0*128:(0+1)*128] = img_left
      imgs[128:256, 1*128:(1+1)*128] = img_right
    imgs = np.reshape(imgs, (1, 256, 256, 1))
    imgs = np.clip(imgs, 0, 255)
    y = np.load(folder + "/out.npy")
    cc = cc+1

    yield (imgs/255), np.array([y])

0


ValueError: ignored

In [ ]:
print(len(folders_all))

In [ ]:
ds_series = tf.data.Dataset.from_generator(
    rand_img, 
    output_types=(tf.float32, tf.int32), 
    output_shapes=((1, 256, 256, 1), (1, 8)))
ds_series.batch(1)

In [ ]:
x, y = next(iter(ds_series.take(1)))
plt.figure(figsize = (5,5))
plt.imshow(x[0,:, :, 0], interpolation='nearest', cmap='gray')
print(y)

In [ ]:
def getModel(input_shape):
  model = Sequential()
  
  model.add(Input(shape = input_shape))
  model.add(Conv2D(32, kernel_size=(3, 3), activation=tf.nn.leaky_relu, padding="same")) 
  model.add(MaxPooling2D(pool_size=(2, 2))) #128, 128
  model.add(Conv2D(64, kernel_size=(3, 3), activation=tf.nn.leaky_relu, padding="same")) 
  model.add(MaxPooling2D(pool_size=(2, 2))) #64, 64
  model.add(Conv2D(128, kernel_size=(3, 3), activation=tf.nn.leaky_relu, padding="same")) 
  model.add(MaxPooling2D(pool_size=(2, 2))) # 32, 32
  model.add(Conv2D(256, kernel_size=(3, 3), activation=tf.nn.leaky_relu, padding="same")) 
  model.add(MaxPooling2D(pool_size=(2, 2))) #16, 16
  model.add(Conv2D(512, kernel_size=(3, 3), activation=tf.nn.leaky_relu, padding="same")) 
  model.add(MaxPooling2D(pool_size=(2, 2))) #8, 8
  model.add(Conv2D(512, kernel_size=(3, 3), activation=tf.nn.leaky_relu, padding="same")) # maybe change this value to be 1024
  model.add(MaxPooling2D(pool_size=(2, 2))) #4, 4
  model.add(Flatten())
  model.add(Dropout(0.5))
  model.add(Dense(50, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(50, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(50, activation='relu'))
  #model.add(Dense(3, activation='softmax'))
  model.add(Dense(8, activation='sigmoid'))
  #model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])
  model.compile(optimizer=tf.keras.optimizers.Adam(lr = 0.00001), loss="binary_crossentropy", metrics=["accuracy"])
  model.build()
  return model

In [ ]:
model = getModel((256, 256, 1))
tf.keras.utils.plot_model(model, show_shapes=True, dpi=64)
model.summary()

In [ ]:
filepath="/content/drive/My Drive/PD/model_8_classes_3/weights/weights-improvement-{epoch:02d}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
logs = tf.keras.callbacks.TensorBoard(log_dir='/content/drive/My Drive/PD/model_8_classes_3/logs')
early = tf.keras.callbacks.EarlyStopping(patience=100, monitor = "val_accuracy")
update = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.8, patience=10, min_lr=0.0001)
callbacks_list = [checkpoint, logs, early, update]

In [ ]:
#model.load_weights("/content/drive/MyDrive/PD/model_8_classes_3/weights/weights-improvement-01.hdf5")
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('/content/drive/My Drive/HVD_APP/classifier.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
x_val = np.zeros((len(folders_valid), 256, 256, 1))
y_val = np.zeros((len(folders_valid), 8))

ii= 0
for folder in folders_valid:
  y_val[ii,:] = np.load(folder + "/out.npy")
  for view_ in [["/front.tif", 0, 0], ["/back.tif", 0, 1], ["/left.tif", 1,0], ["/right.tif", 1, 1]]:
    row = view_[1]
    col = view_[2]
    view = view_[0]
    img = np.array(Image.open(folder + view))/255
    x_val[ii, row*128: row*128+128, col*128: col*128 + 128, 0] = img     
  ii = ii+1

In [ ]:
folders_train = random.sample(folders,300)
x_train = np.zeros((len(folders_train), 256, 256, 1))
y_train = np.zeros((len(folders_train), 8))

ii= 0
for folder in folders_train:
  y_train[ii,:] = np.load(folder + "/out.npy")
  for view_ in [["/front.tif", 0, 0], ["/back.tif", 0, 1], ["/left.tif", 1,0], ["/right.tif", 1, 1]]:
    row = view_[1]
    col = view_[2]
    view = view_[0]
    img = np.array(Image.open(folder + view))/255
    x_train[ii, row*128: row*128+128, col*128: col*128 + 128, 0] = img    
  ii = ii+1

In [ ]:
x_plt = np.concatenate((x_train[0:10, ...], x_train[-10::, ...], x_val[0:10, ...], x_val[-10::, ...]), axis = 0)
y_plt = np.concatenate((y_train[0:10, ...], y_train[-10::, ...], y_val[0:10, ...], y_val[-10::, ...]), axis = 0)
plt.figure(figsize=(64, 16))
for i in range(40):
  plt.subplot(2, 20, i + 1)
  plt.imshow(np.clip(x_plt[i,:, :, 0],0,1), cmap = "gray")
  if y_plt[i,0]==1:
    title = "aty"
  elif y_plt[i,1]==1:
    title = "num"
  elif y_plt[i,2]==1:
    title = "occ1"
  elif y_plt[i,3]==1:
    title = "occ2"
  elif y_plt[i,4]==1:
    title = "rhino"
  elif y_plt[i,5]==1:
    title = "temp1"
  elif y_plt[i,6]==1:
    title = "temp2"
  elif y_plt[i,7]==1:
    title = "frontal"
  plt.title(title)
  plt.axis('off')

In [ ]:
val_accuracy = []
test_accuracy = []
train_accuracy = []

In [ ]:
#val_accuracy = []
#test_accuracy = []
#train_accuracy = []
plt.figure()
for ii in range(25):
  model.fit(rand_img(), validation_data = [x_val, y_val], steps_per_epoch = len(folders), callbacks=callbacks_list, epochs=1)
  train_accuracy.append(accuracy_score(model.predict(x_train)>0.5, y_train))
  val_accuracy.append(accuracy_score(model.predict(x_val)>0.5, y_val))

In [ ]:
  plt.figure()
  plt.plot(val_accuracy, label = "val")
  plt.plot(train_accuracy, label = "train")

In [ ]:

x_plt = x_val[0:60, ...]
y_plt = y_val[0:60, ...]
y_pred = model.predict(x_val)>0.5
plt.figure(figsize=(64, 16))
for i in range(60):
  plt.subplot(3, 20, i + 1)
  plt.imshow(x_plt[i,:, :, 0], cmap = "gray")
  if y_plt[i,0]==1:
    title = "aty"
  elif y_plt[i,1]==1:
    title = "num"
  elif y_plt[i,2]==1:
    title = "occ1"
  elif y_plt[i,3]==1:
    title = "occ2"
  elif y_plt[i,4]==1:
    title = "rhino"
  elif y_plt[i,5]==1:
    title = "temp1"
  elif y_plt[i,6]==1:
    title = "temp2"
  elif y_plt[i,7]==1:
    title = "frontal"

  if y_pred[i,0]==1:
    title = title +"/aty"
  elif y_pred[i,1]==1:
    title = title +"/num"
  elif y_pred[i,2]==1:
    title = title +"/occ1"
  elif y_pred[i,3]==1:
    title = title +"/occ2"
  elif y_pred[i,4]==1:
    title = title +"/rhino"
  elif y_pred[i,5]==1:
    title = title +"/temp1"
  elif y_pred[i,6]==1:
    title = title +"/temp2"
  elif y_pred[i,7]==1:
    title = title +"/frontal"
  plt.title(title)
  plt.axis('off')

In [ ]:
#y_pred = np.array(model.predict(x_val)>0.5).astype(np.uint8)

right = (np.array(y_val) == y_pred)
rec = np.logical_and(right, y_val)
right_guess = np.logical_and(right, y_pred)


print(["aty", "num", "occ1", "occ2", "rhino", "temp1", "temp2", "frontal"])
print("N: ")
print(np.sum(y_val, axis = 0))
print("Accuracy: ")
print(np.sum(right, axis =0)/300)
print("Sensitivity")
print(np.sum(rec, axis =0)/np.sum(y_val, axis = 0))
print("Likelihood")
print(np.sum(right_guess, axis =0)/np.sum(y_pred, axis = 0))


